### Bibliotheques

In [ ]:
#Importation des bibliotheques necessaires
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

import tensorflow_addons as tfa
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D


print("TensorFlow Version:", tf.__version__)

### Data Pre-Processing

In [ ]:
#Dossiers des dataset

base_dir = "../dataset/"
root_dir = "./"
work_dir = root_dir + "adni/"
#work_dir = root_dir + "kaggle/"
    
print("Working Directory Contents:", os.listdir(work_dir))

In [ ]:
#Initialisation des classes et la taille des images
WORK_DIR = './adni/'

CLASSES = ['AD','MCI','CN']

IMG_SIZE = 176
IMAGE_SIZE = [176, 176]
DIM = (IMG_SIZE, IMG_SIZE)

In [ ]:
#Augmentation des données

ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = False
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)
#work_dr = IDG(brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)
train_data_gen = work_dr.flow_from_directory(directory=WORK_DIR, target_size=DIM, batch_size=6500, shuffle=True)

In [ ]:
#Normalisation
image_pixels = train_data.reshape(-1, train_data.shape[-1])

# Creation MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(image_pixels)

# Transformer les pixels en [0, 1]
image_pixels = scaler.transform(image_pixels)

# Retoure au demention originale des images
train_data = image_pixels.reshape(train_data.shape)

In [ ]:
#Affichage des images irm
"""def show_images(generator,y_pred=None):
    
    Input: An image generator,predicted labels (optional)
    Output: Displays a grid of 9 images with lables
    
    
    # get image lables
    labels =dict(zip([0,1,2,3], CLASSES))
    
    # get a batch of images
    x,y = generator.next()
    
    # display a grid of 9 images
    plt.figure(figsize=(10, 10))
    if y_pred is None:
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            idx = randint(0, 6400)
            plt.imshow(x[idx])
            plt.axis("off")
            plt.title("Class:{}".format(labels[np.argmax(y[idx])]))
                                                     
    else:
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(x[i])
            plt.axis("off")
            plt.title("Actual:{} \nPredicted:{}".format(labels[np.argmax(y[i])],labels[y_pred[i]]))
    
# Display Train Images
show_images(train_data_gen)"""

In [ ]:
#Récupérer les données à partir de l'itérateur ImageDataGenerator.

train_data, train_labels = train_data_gen.next()

In [ ]:
#Les dimensions des données

print(train_data.shape, train_labels.shape)

In [ ]:
#Effectuer une sur-échantillonnage (OverSampling) des données, car les classes sont déséquilibrées.

sm = SMOTE(random_state=42)

train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)

train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

print(train_data.shape, train_labels.shape)

In [ ]:
# Verification des données aprés l'equilibre
class_indices = np.argmax(train_labels, axis=1)

# Nombre de classes
class_counts = {cls: np.sum(class_indices == idx) for idx, cls in enumerate(CLASSES)}

# Extraire les noms des classes
counts = list(class_counts.values())
class_names = list(class_counts.keys())

# Creation de graphe
plt.figure(figsize=(10, 6))
plt.bar(class_names, counts, color='green')
plt.xlabel('Classes')
plt.ylabel('Number of Images')
plt.title('Number of Images in Each Class')
plt.xticks(rotation=45, ha='right')


plt.tight_layout()
plt.show()

In [ ]:
#Diviser les données en ensembles d'entraînement, de test et de validation.

train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

### Utilisation de VGG19

In [ ]:
#Importer le modele pre entraine VGG19 (import weights)
vgg_model = VGG19(input_shape=(176, 176, 3), include_top=False, weights="imagenet")

In [ ]:
#Importer le modele pre entraine InceptionV3 (import weights)
#inception_model = InceptionV3(input_shape=(176, 176, 3), include_top=False, weights="imagenet")

In [ ]:
plot_model(vgg_model, show_shapes=True, show_layer_names=True)

In [ ]:
# Freeze layers (Pour appliquer le transfer learning)
for layer in vgg_model.layers:
    layer.trainable=False

In [ ]:
# Couche entierement connecté (Fully connected layers)
custom_vgg_model = Sequential([
        vgg_model,
        
        GlobalAveragePooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        
        Dense(3, activation='softmax')        
    ], name = "vgg_cnn_model")

In [ ]:
#Les fonction de callbacks pour l'entrainement

class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') > 0.99:
            print("\nReached accuracy threshold! Terminating training.")
            self.model.stop_training = True
            
my_callback = MyCallback()

#Eviter la Stabilisation d'entrainement et minimum local
rop_callback = ReduceLROnPlateau(monitor="val_loss", patience=3)

In [ ]:
#Les metriques utilisées, optimiseur, fonction de perte
METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc'),
           tfa.metrics.F1Score(num_classes=3)]

CALLBACKS = [my_callback, rop_callback]
    
custom_vgg_model.compile(optimizer='rmsprop',
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)

custom_vgg_model.summary()

In [ ]:
#L'entrainement de modele en 80 epochs avec les données d'entrainement et validation
EPOCHS = 80

history = custom_vgg_model.fit(train_data, train_labels, validation_data=(val_data, val_labels), callbacks=CALLBACKS, epochs=EPOCHS)

### Test et resultats de modele

In [ ]:
#Les graphe des metriques durant l'entrainement

fig, ax = plt.subplots(1, 3, figsize = (30, 5))
ax = ax.ravel()

for i, metric in enumerate(["acc", "auc", "loss"]):
    ax[i].plot(history.history[metric])
    ax[i].plot(history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("Epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])

In [ ]:
#Evaluation de modele par accuraccy de test

#train_scores = model.evaluate(train_data, train_labels)
#val_scores = model.evaluate(val_data, val_labels)
test_scores = custom_inception_model.evaluate(test_data, test_labels)

#print("Training Accuracy: %.2f%%"%(train_scores[1] * 100))
#print("Validation Accuracy: %.2f%%"%(val_scores[1] * 100))
print("Testing Accuracy: %.2f%%"%(test_scores[1] * 100))

In [ ]:
#Prediction des données de test

pred_labels = custom_inception_model.predict(test_data)

In [ ]:
#Le rapport de classification : precision, reccall, f1 score

def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

print(classification_report(test_labels, pred_labels, target_names=CLASSES))

In [ ]:
#La matrice de confusion
pred_ls = np.argmax(pred_labels, axis=1)
test_ls = np.argmax(test_labels, axis=1)
conf_matrix = confusion_matrix(test_ls, pred_ls)

# Normalize 
conf_matrix_norm = conf_matrix / conf_matrix.sum(axis=1)[:,np.newaxis] 

plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
ax = sns.heatmap(conf_matrix_norm, 
                cmap='Blues', 
                annot=True, 
                fmt='.2%',
                xticklabels=CLASSES,
                yticklabels=CLASSES)

#plt.title('Normalized Confusion Matrix')
plt.xlabel('Classe prédite')
plt.ylabel('Classe réelle')
plt.show()

In [ ]:
#Autre metriques de classification 

print("Balanced Accuracy Score: {} %".format(round(BAS(test_ls, pred_ls) * 100, 2)))
print("Matthew's Correlation Coefficient: {} %".format(round(MCC(test_ls, pred_ls) * 100, 2)))

In [ ]:
#Sauvgarde de modele pour futur utilisation

custom_vgg_model_dir = work_dir + "alzheimer_vgg_cnn_model"
custom_vgg_model.save(custom_vgg_model_dir, save_format='h5')
os.listdir(work_dir)

In [ ]:
#Load le modele entrainé
pretrained_model = tf.keras.models.load_model(custom_vgg_model_dir)

#Verification de l'architecture
plot_model(pretrained_model, to_file=work_dir + "model_plot.png", show_shapes=True, show_layer_names=True)
